# A demo for Gaussian output analyzer

In [ ]:
import os
import sys
# To add this RDMC into PYTHONPATH in case you haven't do it
sys.path.append(os.path.dirname(os.path.abspath('')))

from rdmc.external.gaussian import GaussianLog
from rdmc.view import mol_viewer, grid_viewer

def general_info(glog):
    print(f'Success?: {glog.success}')
    print(f'TS?: {glog.is_ts}')
    print(f'Involved job types: {", ".join(glog.job_type)}')
    try:
        print(f'Scanning: {", ".join(glog.scan_names)}')
    except:
        pass
    print(f'Charge: {glog.charge}, Multiplicity: {glog.multiplicity}')

%load_ext autoreload
%autoreload 2
%matplotlib inline

## Input the path of log file <a id='HOME'></a>
You need to assign the path to `log`. Some examples are provided. 

In [ ]:
############ EXAMPLES #################
# CBS-QB3 non-TS
log = 'data/well-cbsqb3.out'
# CBS-QB3 TS
# log = 'data/ts-cbsqb3.out'
# scan
# log = 'data/scan.out'
# IRC
# log = 'data/irc.out'
######################################

glog = GaussianLog(log)
general_info(glog)

## Hyperlinks: Analyze by job types

- [Optimization](#OPT)
- [Frequency](#FREQ)
- [Scan](#SCAN)
- [IRC](#IRC)

## 1. Optimization <a id='OPT'></a>

### 1.1 Visualize molecule

If the optimization was converged, then show the converged geometry. Otherwise, show the geometry that is the closest to the convergence criteria.

[back](#HOME)

In [ ]:
glog.view_mol(atom_index=True, gv_background=True).update()
xyz = glog.get_best_opt_geom(xyz_str=True)
if glog.success:
    print('Converged XYZ:\n')
else:
    print('Geometry that is the closest to the convergence criteria:\n')
print(xyz)

### 1.2 Convergence analysis

Check the trend for each convergence criterion
- `logy`: plot the y axis in log scale
- `relative`: Plot the relative value to the convergence criteria

[back](#HOME)

In [ ]:
glog.plot_convergence(logy=True, relative=True);

### 1.3 Interact with opt job
[back](#HOME)

In [ ]:
glog.interact_convergence();

## 2. Frequency <a id='FREQ'></a>

### 2.1 Summary 
[back](#HOME)

In [ ]:
print(f'Number of freqs: {glog.freqs.shape[0]}')
print(f'Number of negative freqs: {glog.num_neg_freqs}')
print(f'Negative freqs: {glog.neg_freqs}')

### 2.2 Interact with freq job

- select the frequency you want to visualize
- change its number of frames (the smaller the faster the animation is; the higher the more detailed the animation is)
- change the amplitude of the mode

[back](#HOME)

In [ ]:
glog.interact_freq();

### 2.3 Guess reaction from the imaginary frequency

Guess the reactants and the products from the imaginary frequency mode. This requires the frequency job involves a Transition state molecule. Please be cautious that this method is not very accurate.

- `amplitude`: The amplitude factor on the displacement matrix to generate theguess geometry for the reactant and the product. A smaller factor makes the geometry close to the TS, while a wildly large factor makes the geometry nonphysical.
- `as_mol_frags`: Whether to return the results as molecule complexes or as individual molecules. `False` means return as the complexes.
- `inverse`: Inverse the sequence of the reactant and the product.

There will be messages about SaturateMol, you want to make sure the cell generate `r_mol` and `p_mol` has no failure, while ignore the failure message in the other cell. You may also ignore the SMILES generated for the TS

[back](#HOME)

In [ ]:
r_mols, p_mols = glog.guess_rxn_from_freq(amplitude=[0.1, 0.25], atom_weighted=True, inverse=True)
print(f'{len(r_mols)} potential reactants and {len(p_mols)} potential products are identified.')

In [ ]:
r_idx = 0
p_idx = 0


ts = glog.get_mol(embed_conformers=False)
mols_to_view = [ts]
if r_idx < len(r_mols):
    mols_to_view.insert(0, r_mols[r_idx])
if p_idx < len(p_mols):
    mols_to_view.append(p_mols[p_idx])

entries = len(mols_to_view)
viewer = grid_viewer(viewer_grid=(1, entries), viewer_size=(300 * entries, 300),)
for i in range(entries):
    mol_viewer(mols_to_view[i], viewer=viewer, viewer_loc=(0, i))

print('\nreactant    TS      product')
print([mol.ToSmiles() for mol in mols_to_view])
viewer.show()

## 3. Scan <a id='SCAN'></a>

### 3.1 Visualize the scan
- `align_scan`: if align the scanned coordinate to make the animaiton cleaner
- `align_frag_idx`: which part of the body to align (value should be either 1 or 2)
- `atom_index`: whether to show the atom index

[back](#HOME)

In [ ]:
glog.view_traj(align_scan=True, align_frag_idx=1, atom_index=False, backend='jensen').update();

### 3.2 Plot the scan energies

- `converged`: only plot energies for converged geometries
- `relative_x`: plot the x-axis as a relative value (value for initial geom mas the baseline)
- `relative_y`: plot the y-axis as a relative value (value for initial geom as the baseline)

[back](#HOME)

In [ ]:
glog.plot_scan_energies(converged=True, relative_x=True, relative_y=True);

### 3.3 Interact with the scan job

[back](#HOME)

In [ ]:
glog.interact_scan(align_scan=True, align_frag_idx=1, backend='jensen');

## 4. IRC <a id='IRC'></a>

### 4.1 Visualize the IRC
- `loop`: the way animation plays (`'backAndForth'`, `'forward'`, `'reverse'`)
- `interval`: the time interval between each frame (the smaller the fast the animiation is)

Note: you don't need to worry about the comment of `SaturateMol` failure since we are dealing with TS.

[back](#HOME)

In [ ]:
glog.view_traj(animate={"loop": "backAndForth", "interval": 100}).update();

### 4.2 Plot the IRC energies
- `converged`: only display the energies for the converged geometries

[back](#HOME)

In [ ]:
glog.plot_irc_energies(converged=True);

### 4.3 Interact with the IRC job

[back](#HOME)

In [ ]:
glog.interact_irc();

### 4.4 Guess the reaction

Guess the reactants and the products from the IRC results. This requires the IRC job to bidirectional.
- `index`: the index of the conformer pair that is the distance from the TS. To use the geometries at both ends of the IRC curve, you can assign it to `0`.
- `as_mol_frags`: Whether to return the results as molecule complexes or as individual molecules. `False` means return as the complexes.
- `inverse`: Inverse the sequence of the reactant and the product.

There will be messages about SaturateMol, you want to make sure the cell generate `r_mol` and `p_mol` has no failure, while ignore the failure message in the other cell. You may also ignore the SMILES generated for the TS

[back](#HOME)

In [ ]:
r_mol, p_mol = glog.guess_rxn_from_irc(index=0, as_mol_frags=False, inverse=False)

In [ ]:
ts = glog.get_mol(embed_conformers=False)

if not isinstance(r_mol, tuple):
    r_mol, p_mol = (r_mol,), (p_mol,)

mols_to_view = list(r_mol) + [ts] + list(p_mol)
entries = len(mols_to_view)
viewer = grid_viewer(viewer_grid=(1, entries), viewer_size=(300 * entries, 300),)
for i in range(entries):
    mol_viewer(mols_to_view[i], viewer=viewer, viewer_loc=(0, i))

print('\nreactant    TS      product')
print([mol.ToSmiles() for mol in mols_to_view])
viewer.show()